
# **Homework 3 - What is the best anime in the world?**

**#Group 18**

*Francesca Possenti | Davide Cacciatore | Braulio Villalobos-Quiros*

In [2]:
# Import some functions
import os
import time
import multiprocessing as mp
import ctypes
import nltk
from nltk.stem import PorterStemmer

# Import modules from external .py files
import list_urls as l_u
import content_html as c_html
import anime_information as a_info
import vocabularize as voc
import queries as q

## *1* **Data Collection**

### *1.1* **Get the list of animes**
We start from the list of animes to include in our corpus of documents. In particular, we focus on the top animes listed in the first 400 pages. From this list we want to collect the url associated to each anime in the list. The list is long and splitted in many pages. We ask you to retrieve only the urls of the animes listed in the first 400 pages (each page has 50 animes so you will end up with 20000 unique anime urls).
The output of this step is a .txt file whose single line corresponds to an anime's url.

We extract from the main pages - the first 400 according to pagination - the urls of animes' web pages that will be retrieved in the next points.

In [3]:
path = os.getcwd()
# Get the txt file with all the urls
#l_u.parallelize_extraction()

### *1.2* **Crawl animes**
Once you get all the urls in the first 400 pages of the list, you:

* Download the html corresponding to each of the collected urls.
* After you collect a single page, immediately save its `html` in a file. In this way, if your program stops, for any reason, you will not lose the data collected up to the stopping point.
* Organize the entire set of downloaded `html` pages into folders. Each folder will contain the `htmls` of the animes in page 1, page 2, ... of the list of animes.

**Get the html files from animes' urls previuosly collected**

We download - via urls from the previuos point - and memorize all the html files in a directory organized as follows:
* a new **dir** `page_i` is created for each page, where is collected each `html` of the relative web page.

In [ ]:
# Get the html files
c_html.get_content()

### *1.3* **Parse downloaded pages**
At this point, you should have all the html documents about the animes of interest and you can start to extract the animes informations. 

**Get the needed information about animes**

Here we read each `html` files previously collected to get the requested information from the web pages and we create `tsv` files to memorize them. Each directory - so each page - contains a directory where all the `tsv` of the corresponding `html` files is memorized.

In [ ]:
# Get the info about the animes
a_info.parallelize_parsing(path)

## *2* **Search Engine**

### *2.1* **Conjunctive query**

#### *2.1.1* **Create your index!**

After collecting the necessary data, we started focusing on the search engine.
We initialized some needed objects and files in **JSON** format.

In [4]:
# Initialize object that will be needed: shared 'managed' dictionaries
manager = mp.Manager()
vocabulary = manager.dict()
inverted_index = manager.dict()
complex_index = manager.dict()
docs_short = manager.dict()

# Shared 'managed' counter and lock, useful to control tasks and incrementing the counter properly
v = manager.Value(ctypes.c_ulonglong, 0)
lock = manager.Lock()

# Stemming utilities
nltk.download('punkt')
porter = PorterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


At this point, we computed some `JSON` files that we will use in the next points:

* `vocabulary.txt`: a *txt* file whose lines contain a pair, a pre-processed word and its unique code identifier;
    
* `inverted_index.json`: a *JSON* file whose line contain a pair, a word ID and a list of documents in which the word is present. The documents are represented as a string composed by the string document and their identification number;
    
* `tf_complex_index.json`: similar to *inverted_index*, but the list of documents contains tuples (document, *tf index*). At this point we only have the *tf* - we will get the *Idf* after.

In [ ]:
# Get JSON files for further calculations
voc.parallelize_process_anime(path, vocabulary, inverted_index, complex_index, porter, v, lock)
voc.write_index(vocabulary, "vocabulary")
voc.write_index(inverted_index, "inverted_index")
voc.write_index(complex_index, "tf_complex_index")

Now we compute two other documents which are needed to get the **cosine-similarity**: 

* `tfIdf_complex index.json`: as the *tf_complex_index*, but with the *tfIdf index* instead of the *tf* one. Having two step was necessary since, in order to compute the *Idf index*, we needed some overall information, which was previously missing. After we got the information (stored in the *inverted_index.json*), the rest could be done;
    
* `docs_short.json`: a complementary `json` file needed to compute the custom measure.

In [ ]:
# At this point we have an incomplete complex_index since the number associated to each doc is only the tf part
# Get complete tfIdf complex_index
voc.get_complex_index(complex_index)
voc.write_index(complex_index, "tfIdf_complex_index")
# Get index with documents as key to retrieve information on members/popularity of each anime
voc.parallelize_docs_short(path, docs_short, porter)
voc.write_index(docs_short, "docs_short")

#### *2.1.2* **Execute the query**

In [4]:
# Run conjunctive query and print the result in a table format
docs = q.conjunctive_query(porter)
q.print_tables(docs, path, "")

Search keys - conjunctive query : saiyan race


,animeTitle,animeDescription,Url
0,Dragon Ball Z,Five years after winning the World Martial Art...,https://myanimelist.net/anime/813/Dragon_Ball_Z\n
1,Dragon Ball Z Special 1: Tatta Hitori no Saish...,"Bardock, Son Goku's father, is a low-ranking S...",https://myanimelist.net/anime/986/Dragon_Ball_...
2,Dragon Ball Super: Broly,"Forty-one years ago on Planet Vegeta, home of ...",https://myanimelist.net/anime/36946/Dragon_Bal...
3,Dragon Ball Kai,"Five years after the events of Dragon Ball, ma...",https://myanimelist.net/anime/6033/Dragon_Ball...


In [5]:
# Run conjunctive query and print the result in a table format - example 2
docs = q.conjunctive_query(porter)
q.print_tables(docs, path, "")

Search keys - conjunctive query : criminals and detectives


,animeTitle,animeDescription,Url
0,F-Zero: Falcon Densetsu,Rick Wheeler was a police detective who got in...,https://myanimelist.net/anime/3101/F-Zero__Fal...
1,Fuuto Tantei,"""In the city of Fuuto, criminals make use of U...",https://myanimelist.net/anime/48649/Fuuto_Tant...
2,Meitantei Conan: Hannin no Hanzawa-san,"""Anime adaptation of the spin-off manga Meitan...",https://myanimelist.net/anime/50010/Meitantei_...
3,Naikaku Kenryoku Hanzai Kyousei Torishimarikan...,Based on Ken Kitashiba & Yasuhiro Watanabe's p...,https://myanimelist.net/anime/1740/Naikaku_Ken...
4,Yuukoku no Moriarty 2nd Season,"""Great Britain is ablaze with news of a so-cal...",https://myanimelist.net/anime/43325/Yuukoku_no...
5,Cuticle Tantei Inaba,"In a world where half-human, half-animal chime...",https://myanimelist.net/anime/15109/Cuticle_Ta...
6,Kamisama no Memochou,"""Narumi Fujishima may seem like a normal high ...",https://myanimelist.net/anime/10568/Kamisama_n...
7,Hurricane Polymar,Yoroi Takeshi is a carefree and somewhat naïve...,https://myanimelist.net/anime/4808/Hurricane_P...
8,Eightman After,Private detective Hazama Itsuru is hired by Pr...,https://myanimelist.net/anime/1344/Eightman_Af...
9,Death Note,"A shinigami, as a god of death, can kill any p...",https://myanimelist.net/anime/1535/Death_Note\n


In [6]:
# Run conjunctive query and print the result in a table format - example 3
docs = q.conjunctive_query(porter)
q.print_tables(docs, path, "")

Search keys - conjunctive query : parents


,animeTitle,animeDescription,Url
0,Super Cub,"Koguma, a reserved second-year high school stu...",https://myanimelist.net/anime/40685/Super_Cub\n
1,Mahoutsukai Sally 2,During Sally's coronation ceremony in the magi...,https://myanimelist.net/anime/3875/Mahoutsukai...
2,Nezha Zhi Mo Tong Jiang Shi,From the heavenly object known as the Chaos Pe...,https://myanimelist.net/anime/40111/Nezha_Zhi_...
3,Hitozuma Ryoujoku Sankanbi,Three wives are required to attend a parent/te...,https://myanimelist.net/anime/2945/Hitozuma_Ry...
4,Mekakucity Records,"""Mekakucity Records follows Mekakucity Days, a...",https://myanimelist.net/anime/36494/Mekakucity...
...,...,...,...
341,DC Super Heroes vs Taka no Tsume-dan,Joker has come to Japan to steal the Eagle Tal...,https://myanimelist.net/anime/35219/DC_Super_H...
342,Bokura wa Minna Kawai-sou,Kazunari Usa is a high school freshman who wil...,https://myanimelist.net/anime/21405/Bokura_wa_...
343,Kotarou wa Hitorigurashi,"""The """"apartment comedy with laughs and tears""...",https://myanimelist.net/anime/49909/Kotarou_wa...
344,Yume no Crayon Oukoku,The civilians of the Crayon Kingdom have alway...,https://myanimelist.net/anime/4651/Yume_no_Cra...


### *2.2* **Conjunctive query & Ranking score**

We compute the *tfIdf* score and the *cosine similarity*.

The **cosine similarity** is given by:

$cos(q, d^{i}) = \frac{\sum_{j=1}^{d}{tfIdf_{ij}}}{||d^{i}||_{2}}$

where $q$ is the provided query, $d^{i}$ is the $i^{th}$ document and $j$ are the words in the query that are in the document.



In [5]:
# Run conjunctive query and print the ranked results as table
# We choose to print the top 5 results
top = q.cosine_similarity_rank(porter, 5)
q.print_tables(top, path, "tfIdf")

Search keys - conjunctive query : saiyan race


,animeTitle,animeDescription,Url,Similarity
0,Dragon Ball Z Special 1: Tatta Hitori no Saish...,"Bardock, Son Goku's father, is a low-ranking S...",https://myanimelist.net/anime/986/Dragon_Ball_...,0.21109
1,Dragon Ball Super: Broly,"Forty-one years ago on Planet Vegeta, home of ...",https://myanimelist.net/anime/36946/Dragon_Bal...,0.06115
2,Dragon Ball Z,Five years after winning the World Martial Art...,https://myanimelist.net/anime/813/Dragon_Ball_Z\n,0.05138
3,Dragon Ball Kai,"Five years after the events of Dragon Ball, ma...",https://myanimelist.net/anime/6033/Dragon_Ball...,0.04113


In [6]:
# Run conjunctive query and print the ranked results as table - example 2
# We choose to print the top 5 results
top = q.cosine_similarity_rank(porter, 5)
q.print_tables(top, path, "tfIdf")

Search keys - conjunctive query : criminals and detectives


,animeTitle,animeDescription,Url,Similarity
0,Meitantei Conan: Hannin no Hanzawa-san,"""Anime adaptation of the spin-off manga Meitan...",https://myanimelist.net/anime/50010/Meitantei_...,0.16008
1,Naikaku Kenryoku Hanzai Kyousei Torishimarikan...,Based on Ken Kitashiba & Yasuhiro Watanabe's p...,https://myanimelist.net/anime/1740/Naikaku_Ken...,0.12768
2,Death Note,"A shinigami, as a god of death, can kill any p...",https://myanimelist.net/anime/1535/Death_Note\n,0.12545
3,Kamisama no Memochou,"""Narumi Fujishima may seem like a normal high ...",https://myanimelist.net/anime/10568/Kamisama_n...,0.12141
4,F-Zero: Falcon Densetsu,Rick Wheeler was a police detective who got in...,https://myanimelist.net/anime/3101/F-Zero__Fal...,0.10446


In [7]:
# Run conjunctive query and print the ranked results as table - example 3
# We choose to print the top 5 results
top = q.cosine_similarity_rank(porter, 5)
q.print_tables(top, path, "tfIdf")

Search keys - conjunctive query : parents


,animeTitle,animeDescription,Url,Similarity
0,Ultraman Kids: Haha wo Tazunete 3000-man Kounen,Ultraman Kids go on a journey to search for th...,https://myanimelist.net/anime/8753/Ultraman_Ki...,0.33387
1,Kateikyoushi no Oneesan 2 The Animation: H no ...,Touya's days with his tutors continue as his p...,https://myanimelist.net/anime/9311/Kateikyoush...,0.31856
2,Ijou Chitai: Jikken Dorei,"After her parents died, Konno Azusa inherits h...",https://myanimelist.net/anime/11815/Ijou_Chita...,0.27588
3,Watashi to Watashi: Futari no Lotte,Outspoken Luise and shy Lotte meet at summer c...,https://myanimelist.net/anime/3691/Watashi_to_...,0.27531
4,Mamoru Hikitsugu Watashitachi no Kurashi,A young man who has left his parents' home fol...,https://myanimelist.net/anime/42507/Mamoru_Hik...,0.26271


## *3* **Define a new score!** 

As our custom scoring function, we decided to implement the following formula:

$cos(q, d^{i})*0.5+(1-\frac{p^{i}}{\max(p)})*0.25+(\frac{m^{i}}{\max(m)})*0.25$

Where $p^{i}$ is the **popularity** value of the $i^{th}$ anime and $m^{i}$ is the number of **members** that the $i^{th}$ anime has gotten.

With this function we get a new index that is a weighted average of different values. This index doesn't take into account only the cosine similarity but also a *'popularity index'* and a *'members index'*.

* The *cosine similarity*, that goes from 0 to 1, has weight 0.5;
* The *popularity index* $(1-\frac{p^{i}}{\max(p)})$ assigns 0 to the less popular anime and 1 to the most popular one. This index has weight 0.25;
* The *members index* $(\frac{m^{i}}{\max(m)})$ assigns 0 to the anime with the lowest number of members and 1 to the anime with the highest number.

In this way, our new **ranking system** takes care also of the popularity of the anime and the number of members it has. So, the top-k documents that we show are not only the most similar to the query but also the most famous animes that are more likely to be searched.



In [8]:
# Run conjunctive query and print the ranked results as table
top = q.custom_rank(porter, 5)
q.print_tables(top, path, "tfIdf")

Search keys - custom query : saiyan race


,animeTitle,animeDescription,Url,Similarity
0,Dragon Ball Z,Five years after winning the World Martial Art...,https://myanimelist.net/anime/813/Dragon_Ball_Z\n,0.30492
1,Dragon Ball Kai,"Five years after the events of Dragon Ball, ma...",https://myanimelist.net/anime/6033/Dragon_Ball...,0.24735
2,Dragon Ball Super: Broly,"Forty-one years ago on Planet Vegeta, home of ...",https://myanimelist.net/anime/36946/Dragon_Bal...,0.23324
3,Dragon Ball Z Special 1: Tatta Hitori no Saish...,"Bardock, Son Goku's father, is a low-ranking S...",https://myanimelist.net/anime/986/Dragon_Ball_...,0.12806


The results are quite different from the last query. 

Now, the first anime is **Dragon Ball Z**, because even though it doesn't have an high cosine similarity with the query, it is the most famous anime related to *'sayian race'*. **Dragon Ball Super: Broly** is now the third placed, due to the fact that is less famous than the others.

We think that these results can be more useful to the users, because we show them first the most famouses anime related with the query and not the ones that are similar. The most famouses anime are more likely to be searched and they attract more people.

In [9]:
# Run conjunctive query and print the ranked results as table - example 2
top = q.custom_rank(porter, 5)
q.print_tables(top, path, "tfIdf")

Search keys - custom query : criminals and detectives


,animeTitle,animeDescription,Url,Similarity
0,Death Note,"A shinigami, as a god of death, can kill any p...",https://myanimelist.net/anime/1535/Death_Note\n,0.43726
1,Yuu☆Yuu☆Hakusho,"One fateful day, Yuusuke Urameshi, a 14-year-o...",https://myanimelist.net/anime/392/Yuu☆Yuu☆Haku...,0.25212
2,Detective Conan,"Shinichi Kudou, a high school student of astou...",https://myanimelist.net/anime/235/Detective_Co...,0.21890
3,Id:Invaded,"""The Mizuhanome System is a highly advanced de...",https://myanimelist.net/anime/40046/Id_Invaded\n,0.21248
4,Yuukoku no Moriarty 2nd Season,"""Great Britain is ablaze with news of a so-cal...",https://myanimelist.net/anime/43325/Yuukoku_no...,0.20477


In [10]:
# Run conjunctive query and print the ranked results as table - example 3
top = q.custom_rank(porter, 5)
q.print_tables(top, path, "tfIdf")

Search keys - custom query : parents


,animeTitle,animeDescription,Url,Similarity
0,FLCL,Naota Nandaba is an ordinary sixth grader livi...,https://myanimelist.net/anime/227/FLCL\n,0.28813
1,Mushoku Tensei: Isekai Ittara Honki Dasu,"Despite being bullied, scorned, and oppressed ...",https://myanimelist.net/anime/39535/Mushoku_Te...,0.28172
2,Monster Musume no Iru Nichijou,"""With his parents abroad, Kimihito Kurusu live...",https://myanimelist.net/anime/30307/Monster_Mu...,0.27816
3,Kekkai Sensen,"""Supersonic monkeys, vampires, talking fishmen...",https://myanimelist.net/anime/24439/Kekkai_Sen...,0.27636
4,Eromanga-sensei,"One year ago, Sagiri Izumi became step-sibling...",https://myanimelist.net/anime/32901/Eromanga-s...,0.27356


## *5* **Algorithmic question**

You consult for a personal trainer who has a *back-to-back sequence* of requests for appointment. A sequence of requests is of the form > 30, 40, 25, 50, 30, 20 where each number is the time that the person who makes the appointment wants to spend. You need to accept some requests, however you need a break between them, so you cannot accept two consecutive requests. For example, `[30, 50, 20]` is an acceptable solution (of duration *100*), but `[30, 40, 50, 20]` is not because *30* and *40* are two consecutive appointments. Your goal is to provide to the personal trainer a schedule that maximizes the total lenght of the accepted appointments. For example, in the previous instance, the optimal solution is `[40, 50, 20]`, of total duration *110*.
1. Write an algorithm that computes the acceptable solution with the longest possible duration.
2. Implement a program that given in input an instance in the form given above, gives the optimal solution.

#### *1* Write an algorithm that computes the acceptable solution with the longest possible duration. 


        Input:
            A: array of length n

        function alg(A):
            S <- array of length n to memorize all the sums
            dict = {}
            for i in n:
                S[i] = max(S[i-1], S[i-2] + A[i])
                if max(S[i-1], S[i-2] + A[i]) == (S[i-2] + A[i]):
                    dict[S[i-2], A[i]] = S[i]
            list <- list containing the number of the optimal solution
            individual <- 2nd index of the keys
            cumulative <- 1st index of the keys
            
            Extract all the individual values and 
            list.append(individual)

            return A, list

#### 2. Implement a program that given in input an instance in the form given above, gives the optimal solution.

In [ ]:
def my_schedule(array):
    # Create an array we can use to memorize sums
    S = [0 for j in range(len(array))]
    
    # Create a dictionary to store all the accepted appointments
    accepted = dict()
    
    for i in range(len(array)):
        # Memorize the maximum non consecutive sum between elements in the array
        # It chooses between the last maximum sum and the one updated 
        # with the i-th element.
        S[i] = max(S[i-1], S[i-2] + array[i])
        
        # Add the accepted appointment
        if max(S[i-1], S[i-2] + array[i]) == (S[i-2] + array[i]):
            accepted[S[i-2], array[i]] = S[i]
            
    # Retrieving the numbers that produce the optimal solution
    
    # List that contains the number that produce the optimal solution
    list_of_success = []
    
    # Defining the max value of the dictionary to start the while loop
    max_value = max(accepted.values())
    
    # Defining a stop clause which is the cumulative sum or the 1st index of the keys
    stop_clause = list(accepted.keys())[list(accepted.values()).index(max_value)][0]
    
    # Stop until the cumulative value is equal to zero, which means we finished the list
    while stop_clause != 0:
        
        # Suppose we have an entry of a dictionary that is (130, 110): 240
        # Define the first index of the key, i.e. 130, as the cumulative number
        # Define the second index of the key, i.e. 110, as the individual number
        
        individual_value = list(accepted.keys())[list(accepted.values()).index(max_value)][1]
        cumulative_value = list(accepted.keys())[list(accepted.values()).index(max_value)][0]
        
        # We append the individual value to the list, since this number is part of the optimal solution
        list_of_success.append(individual_value)
        
        # Redefine the max value as
        max_value = cumulative_value
        stop_clause = cumulative_value
        
    # Reversing the list that contains the numbers of the optimal solution,
    # so they appear in the original order.
    list_of_success.reverse()
    
    return S[-1], list_of_success

In [ ]:
appointments = [30, 40, 25, 50, 30, 20]
tot, opt_sol = my_schedule(appointments)
print('The optimal solution of accepted appointments is:', opt_sol)
print('The total duration of appointments is:', tot)

In [ ]:
# Try with different lists of appointments
app1 = [30, 40, 55, 90, 90, 110]
tot, opt_sol = my_schedule(app1)
print('The optimal solution of accepted appointments is:', opt_sol)
print('The total duration of appointments is:', tot)

In [ ]:
app2 = [30, 40, 55, 90, 90, 40]
tot, opt_sol = my_schedule(app2)
print('The optimal solution of accepted appointments is:', opt_sol)
print('The total duration of appointments is:', tot)